In [ ]:
%pip install scholarly tqdm

In [ ]:
from scholarly import scholarly, ProxyGenerator
from tqdm import tqdm
import pickle

# Download


## Publications


In [ ]:
pg = ProxyGenerator()
pg.FreeProxies()
scholarly.use_proxy(pg)

In [ ]:
publication_iterator = scholarly.search_pubs("Artificial Intelligence")

In [ ]:
with open("./data/publication_iterator", "rb") as fp:
    test = pickle.load(fp)

In [ ]:
publications = []

In [ ]:
pg = ProxyGenerator()

while True:
    try:
        publication = next(publication_iterator)
        if not publication:
            break
        publications.append(publication)
        print(len(publications))
    except Exception as exception:
        print(exception)
        pg.FreeProxies()
        scholarly.use_proxy(pg)

In [ ]:
with open("./data/publications", "wb") as fp:
    pickle.dump(publications, fp)

## H-Index


In [ ]:
with open("./data/publications", "rb") as fp:
    publications = pickle.load(fp)

In [ ]:
# pg = ProxyGenerator()

for index, publication in enumerate(tqdm(publications)):
    if len(hindices) - 1 >= index:
        continue

    author_ids = publication["author_id"]
    publication_hindices = []

    for author_id in author_ids:
        if not author_id:
            continue

        successful = False
        while not successful:
            try:
                author = scholarly.search_author_id(author_id)
                scholarly.fill(author, sections=["indices"])
                successful = True
            except Exception as exception:
                print(f"{exception} Retrying...")
                print(publication)
                print(index)
                # pg.FreeProxies()
                # scholarly.use_proxy(pg)

        hindex = author["hindex"]
        hindex5y = author["hindex5y"]
        i10index = author["i10index"]
        i10index5y = author["i10index5y"]

        publication_hindices.append(hindex)

    hindices.append(publication_hindices)

In [ ]:
with open("./data/hindices", "wb") as fp:
    pickle.dump(hindices, fp)

# Cited by


In [ ]:
with open("./data/hindices", "rb") as fp:
    hindices = pickle.load(fp)

with open("./data/publications", "rb") as fp:
    publications = pickle.load(fp)

In [ ]:
with open("./data/citedbys", "rb") as fp:
    citedbys = pickle.load(fp)

In [ ]:
MAX_CITED_BY = 5
pg = ProxyGenerator()

for index, publication in enumerate(tqdm(publications)):
    if len(citedbys) - 1 >= index:
        continue

    successful = False
    while not successful:
        try:
            citedbyListGenerator = scholarly.citedby(publication)
            successful = True
        except Exception as exception:
            print(f"{exception} Retrying...")
            pg.FreeProxies()
            scholarly.use_proxy(pg)

    citedbyList = []

    while len(citedbyList) < MAX_CITED_BY:
        try:
            publication = next(citedbyListGenerator)
            if not publication:
                break
            citedbyList.append(publication)
        except Exception as exception:
            print(exception)
            pg.FreeProxies()
            scholarly.use_proxy(pg)

    citedbys.append(citedbyList)

In [ ]:
with open("./data/citedbys", "wb") as fp:
    pickle.dump(citedbys, fp)

# Cited by H-Index


In [ ]:
with open("./data/citedbys", "rb") as fp:
    citedbys = pickle.load(fp)

In [ ]:
with open("./data/citedbys_hindex", "rb") as fp:
    citedbys_hindex = pickle.load(fp)

In [ ]:
for index, citedby in enumerate(tqdm(citedbys)):
    if len(citedbys_hindex) - 1 >= index:
        continue

    tmp_hindices = []

    for publication in citedby:
        author_ids = publication["author_id"]
        publication_hindices = []

        for author_id in author_ids:
            if not author_id:
                continue

            successful = False
            while not successful:
                try:
                    author = scholarly.search_author_id(author_id)
                    scholarly.fill(author, sections=["indices"])
                    successful = True
                except Exception as exception:
                    print(f"{exception} Retrying...")
                    print(publication)
                    print(index)
                    # pg.FreeProxies()
                    # scholarly.use_proxy(pg)

            hindex = author["hindex"]
            hindex5y = author["hindex5y"]
            i10index = author["i10index"]
            i10index5y = author["i10index5y"]

            publication_hindices.append(hindex)

        tmp_hindices.append(publication_hindices)

    citedbys_hindex.append(tmp_hindices)

In [ ]:
with open("./data/citedbys_hindex", "wb") as fp:
    pickle.dump(citedbys_hindex, fp)

# Data to JS


In [ ]:
with open("./data/citedbys_hindex", "rb") as fp:
    citedbys_hindex = pickle.load(fp)

with open("./data/hindices", "rb") as fp:
    hindices = pickle.load(fp)

In [ ]:
js_string = "const hindices=["

for index, citedby_hindices in enumerate(citedbys_hindex):
    hindex = hindices[index]
    citedby_hindices = sum(citedby_hindices, [])  # ravel

    if len(hindex) == 0 or len(citedby_hindices) == 0:
        continue

    hindex_str = str(hindex)

    js_string += "{"
    js_string += f"self:{hindex_str},"
    js_string += f"citedBy:{citedby_hindices}"
    js_string += "},"

js_string += "];"

with open("./plot/hindices.js", "w") as text_file:
    text_file.write(js_string)